# YOLO11 Training and RKNN Conversion

![frogforce503_logo_2.png](https://frogforce503.org/FFmods/img/frogforce503_logo_2.png)

This notebook trains a YOLO11 model using a Roboflow Dataset and converts it to the RKNN format to run on Rockchip Processors such as the RK3588 found on the Single Board Computer, the Orange Pi 5.

To begin, go to session options on the right panel of the editor and select GPU T4 x2 as your accelerator, and enable using the internet.

You can also upload your own pre-trained YOLO11 model if you trained it using the airockchip/ultralytics_yolo11 repository rather than training a new model.

If you are new to python, object detection, jupyter notebooks, or anything else, check out the helpful links below.

# Helpful Links 😀
Jupyter Notebook Programming:
*   Jupyter - [Jupyter Notebook Documentation](https://jupyter-notebook.readthedocs.io/en/latest/)
*   Kaggle - [How to use Kaggle](https://www.kaggle.com/docs/notebooks)
*   w3schools - [Python Course](https://www.w3schools.com/python/default.asp)
*   Amazon Web Services - [Command Line Interface](https://aws.amazon.com/what-is/cli)
*   Tutorialspoint - [Magic Commands](https://www.tutorialspoint.com/jupyter/ipython_magic_commands.htm)


Machine Learning Fundamentals:
*   IBM - [What is Machine Learning?](https://www.ibm.com/topics/machine-learning)
*   IBM - [Neural Networks](https://www.ibm.com/topics/neural-networks)


About YOLO11:
*   Data Scientist - [What is YOLO?](https://datascientest.com/en/you-only-look-once-yolo-what-is-it)
*   Roboflow - [What is YOLO11?](https://blog.roboflow.com/what-is-yolo11)
*   Ultralytics - [YOLO Docs](https://docs.ultralytics.com/)





About the Orange Pi 5:

*   BAE Systems - [Single Board Computers](https://www.baesystems.com/en-us/definition/what-are-single-board-computers)
*   Orange Pi - [Orange Pi 5 Specs](http://www.orangepi.org/html/hardWare/computerAndMicrocontrollers/details/Orange-Pi-5.html)

# Setup
YOLO11 is a version of the YOLO(You Only Look Once) computer vision model created by Ultralytics. This notebook will create and train a YOLO11 model from scratch.

We are going through a non-standard installation process by cloning the 3rd party ultralytics_yolo11 repository which implements RKNN support in the YOLO11 model.

The official ultralytics YOLO11 model does not have RKNN support, so we can't install the standard stuff.

This is important because in order for the NPUs the Rockchip Processors to run the model, they have to be formatted in RKNN.

In [ ]:
import os #python module that allows for you to interact with the elements of the operating system like directories, processes, environments, paths

#getcwd() gets the working directory we are currently in. We're always working in this directory, we'll store it in avariable called root path
#os.getcwd returns a string
root_path = os.getcwd()

#in a kaggle environment, it should print /kaggle/working
print(root_path)

#cd stands for change directory, we are just moving into the root path directory
#% means we are doing a magic command, % means it works on one line, %% means it works on an entire cell of code. It is used in interactive environments like Jupyter notebooks such as this one.
%cd {root_path}

#! means a shell command which is in command line interface.
!git clone https://github.com/airockchip/ultralytics_yolo11 ultralytics
%cd ultralytics

#installing dependencies
!pip install -e .

#import to use for code
import ultralytics

/content
/content
fatal: destination path 'ultralytics' already exists and is not an empty directory.
/content/ultralytics
Obtaining file:///content/ultralytics
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.3.9-0.editable-py3-none-any.whl size=22989 sha256=634722f04b367568594136e1ba61693b047354a8f8a887e0b3473c1b291af69f
  Stored in directory: /tmp/pip-ephem-wheel-cache-zp059a7l/wheels/ea/71/6b/a9012dfb148489fd8125c2310e565414c996b3c7721defe799
Successfully built ultralytics
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.9
    Uninstalling ultralytics-8.3.9:
      Successfully uninstalled ultralytics-8.3.9


# Downloading a Dataset
[Roboflow Universe](https://universe.roboflow.com/) is an open source repository where you can find datasets to train models you need.

With object detection, a dataset is a collection of images that are annotated with bounding boxes and classifications of objects your machine learning model has to detect.

For this notebook's use, we want to use a YOLO11 object detection model, that means we need to download a dataset from Roboflow that is formatted to be used by the model.

To get a dataset, go to https://universe.roboflow.com/ and find a project. Press datasets on the left bar of the page. Then on the right side, press download dataset. Select a format, for this notebook, select YOLO11 and select show download code. Copy and paste the code snippet in the cell below. You NEED to do this NO MATTER WHAT because each user has their own roboflow API key.

Even if you aren't copying a code snippet, you still need to find your API Key. Go to https://app.roboflow.com/ and log in. In the left navigation bar, press settings. Then press API Keys. You will use your private API Key.

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read(root_path + '/config.ini')
ROBOFLOW_API_KEY = config.get('ROBOFLOW', 'ROBOFLOW_API_KEY')
ROBOFLOW_WORKSPACE = config.get('ROBOFLOW', 'ROBOFLOW_WORKSPACE')
ROBOFLOW_PROJECT = config.get('ROBOFLOW', 'ROBOFLOW_PROJECT')
ROBOFLOW_PROJECT_VERSION = config.get('ROBOFLOW', 'ROBOFLOW_PROJECT_VERSION')
ROBOFLOW_PROJECT_VERSION_DOWNLOAD = config.get('ROBOFLOW', 'ROBOFLOW_PROJECT_VERSION_DOWNLOAD')

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
version = project.version(ROBOFLOW_PROJECT_VERSION)
dataset = version.download(ROBOFLOW_PROJECT_VERSION_DOWNLOAD)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Crime-Detection-1 in yolov11:: 100%|██████████| 27886/27886 [00:05<00:00, 5204.14it/s] 


In [ ]:
#%cat prints the data in a file (referenced by its path)
%cat {dataset.location}/data.yaml

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 2
names: ['knife', 'pistol']

roboflow:
  workspace: proyecto-final-6hd5x
  project: crime-detection-2upma
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/proyecto-final-6hd5x/crime-detection-2upma/dataset/1

### Fixing Dataset Directories

In [ ]:
import yaml

with open(dataset.location + "/data.yaml", 'r') as stream: #open the data.yaml file, 'r' means it is read only
    data = yaml.safe_load(stream) #takes the data from yaml and turns it into dictionaries
    data["train"] = dataset.location + "/train/images" if "train" in data else print("dataset doesn't have training images") #the key 'train' contains the path to the train images, replacing existing path with the correct path
    data["val"] = dataset.location + "/valid/images" if "val" in data else print("dataset doesn't have validating images")
    data["test"] = dataset.location + "/test/images" if "test" in data else print("dataset doesn't have testing images")

with open(dataset.location + "/data.yaml", 'w') as stream: #'w' means we can write in the file
    yaml.dump(data, stream, default_flow_style=False) #writes the data variable into the data.yaml file, default flow style just changes how the data looks in the file

In [ ]:
%cat {dataset.location}/data.yaml

names:
- knife
- pistol
nc: 2
roboflow:
  license: CC BY 4.0
  project: crime-detection-2upma
  url: https://universe.roboflow.com/proyecto-final-6hd5x/crime-detection-2upma/dataset/1
  version: 1
  workspace: proyecto-final-6hd5x
test: /content/ultralytics/Crime-Detection-1/test/images
train: /content/ultralytics/Crime-Detection-1/train/images
val: /content/ultralytics/Crime-Detection-1/valid/images


# Creating a Model

If you have already trained a model using the airockchip/ultralytics_yolo11 repository, and you want to skip the training steps set the boolean `skip_training` variable equal to True.

In [ ]:
skip_training = False

from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def skip(line, cell):
    if not eval(line):
        return get_ipython().run_cell(cell)

## Training Setup

In [ ]:
import os
import re
from IPython.core.magic import register_line_cell_magic

#register_line_cell_magic means we can write our own magic command
@register_line_cell_magic
def writetemplate(line, cell): #takes EVERYTHING in the cell and puts it in the file that's on the same line as when write template is called
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))
        #** means unspecified number of arguments.
        #globals() returns the global symbol table which has the every variable in the notebook.
        #Any variables inside the cell are replaced by their values from the global symbol table using the format function.
        #write writes the entire formatted cell into the file, including comments
        print("Wrote successfully to " + line)

In [ ]:
# define number of classes based on YAML
with open(dataset.location + "/data.yaml", 'r') as stream: #reads the data.yaml file as the variable stream
    #yaml.safeload returns the contents of stream as a dictionary of dictionaries
    num_classes = str(len(yaml.safe_load(stream)['names'])) #number of terms in the "names" key is the number of classes

print(f"num_classes: {num_classes}")
%cd {root_path}/ultralytics

num_classes: 2
/content/ultralytics


### YOLO11 Architecture
Yolo11 architecture found here: https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/models/11/yolo11.yaml.

Creating a custom yaml file to fit the number of classes in our dataset so that we can train models from scratch.

In [ ]:
%%writetemplate custom_yolo11.yaml

# Ultralytics YOLO 🚀, AGPL-3.0 license
# YOLO11 object detection model with P3-P5 outputs. For Usage examples see https://docs.ultralytics.com/tasks/detect

# Parameters
nc: {num_classes}  # number of classes
scales: # model compound scaling constants, i.e. 'model=yolo11n.yaml' will call yolo11.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.50, 0.25, 1024] # summary: 319 layers, 2624080 parameters, 2624064 gradients, 6.6 GFLOPs
  s: [0.50, 0.50, 1024] # summary: 319 layers, 9458752 parameters, 9458736 gradients, 21.7 GFLOPs
  m: [0.50, 1.00, 512] # summary: 409 layers, 20114688 parameters, 20114672 gradients, 68.5 GFLOPs
  l: [1.00, 1.00, 512] # summary: 631 layers, 25372160 parameters, 25372144 gradients, 87.6 GFLOPs
  x: [1.00, 1.50, 512] # summary: 631 layers, 56966176 parameters, 56966160 gradients, 196.0 GFLOPs

# YOLO11n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]] # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]] # 1-P2/4
  - [-1, 2, C3k2, [256, False, 0.25]]
  - [-1, 1, Conv, [256, 3, 2]] # 3-P3/8
  - [-1, 2, C3k2, [512, False, 0.25]]
  - [-1, 1, Conv, [512, 3, 2]] # 5-P4/16
  - [-1, 2, C3k2, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]] # 7-P5/32
  - [-1, 2, C3k2, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]] # 9
  - [-1, 2, C2PSA, [1024]] # 10

# YOLO11n head
head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 2, C3k2, [512, False]] # 13

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 2, C3k2, [256, False]] # 16 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 13], 1, Concat, [1]] # cat head P4
  - [-1, 2, C3k2, [512, False]] # 19 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 10], 1, Concat, [1]] # cat head P5
  - [-1, 2, C3k2, [1024, True]] # 22 (P5/32-large)

  - [[16, 19, 22], 1, Detect, [nc]] # Detect(P3, P4, P5)

Wrote successfully to custom_yolo11.yaml


## Training
You can adjust the following settings:

1.   model: one of [yolo11n, yolo11s, yolo11m, yolo11l, yolo11x], yolo11n is recommended for a Orange Pi 5.
2.   image_size: The input size of the images fed to the model. Should be a multiple of 32.
4.   epochs: How many times the model goes through ALL the data

View other arguments for the train method here: https://docs.ultralytics.com/modes/train/#train-settings

You can see the progress and metrics in the console below the cell. Your cls_loss, box_loss, and dfl_loss should be minimized (less than 1) and your P (precision), R (recall), mAP50, and mAP50-95 should be maximized (as close to 1.00 as possible).

In [ ]:
%cd {root_path}/ultralytics

import os
os.environ['WANDB_DISABLED'] = 'true'

image_size = 448 #640
model = "yolo11n"

/content/ultralytics


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
%%skip skip_training

!yolo task=detect mode=train model=./custom_{model}.yaml data={dataset.location}/data.yaml epochs=20 imgsz={image_size} device=cuda:0 batch = 200


New https://pypi.org/project/ultralytics/8.3.110 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.9 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=./custom_yolo11n.yaml, data=/content/ultralytics/Crime-Detection-1/data.yaml, epochs=20, time=None, patience=100, batch=200, imgsz=448, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, 

<ExecutionResult object at 79ca042b3950, execution_count=None error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 79ca042b04d0, raw_cell="
!yolo task=detect mode=train model=./custom_{mode.." store_history=False silent=False shell_futures=True cell_id=None> result=None>

## Locating trained model

In [ ]:
latest_modified_time = 0
latest = None

#gets the path of the trained model by going through all of folder, subfolders, and files, and finding the file: best.pt
for foldername, subfolders, filenames in os.walk(root_path):
    for filename in filenames:
        if filename == "best.pt":
            file_path = os.path.join(foldername, filename)
            modified_time = os.path.getmtime(file_path)
            if modified_time > latest_modified_time:
                latest_modified_time = modified_time
                latest = file_path
print(latest)

/content/ultralytics/runs/detect/train/weights/best.pt


## Model Name

If you are using an rknn model with PhotonVision for FRC, your model name should be {name}-{horizontal_resolution}-{vertical_resolution}-{model_type}. Name should NOT have dashes, model_type is yolov11n, yolov11s, yolov11m, etc.

Modify the `model_name` variable to match your desired name, DO NOT PUT '.pt or .rknn' in the name

If you leave `model_name` equal to **None**, still run the code cell below. The code will automatically generate a name that fits model naming criteria PhotonVision's criteria

In [ ]:
model_name = None

## Moving and Renaming Model

In [ ]:
%%skip skip_training
import os

if model_name == None:
    dataset_name = dataset.location.replace(f"{root_path}/ultralytics/", "").replace(" ", "_").replace("-", "_")
    model_name = f"{dataset_name}-{image_size}-{image_size}-{model[:4] + 'v' + model[4:]}" #yolo11n -> yolov11n

if latest == None:
    latest = f"{root_path}/{model_name}.pt"
    os.rename(latest, f"{root_path}/{model_name}.pt")
    latest = f"{root_path}/{model_name}.pt"
else:
    os.rename(latest, f"{root_path}/{model_name}.pt")
    latest = f"{root_path}/{model_name}.pt"
print(latest)

/content/Crime_Detection_1-448-448-yolov11n.pt


<ExecutionResult object at 79ca0615ed90, execution_count=None error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 79ca06a9de90, raw_cell="import os

if model_name == None:
    dataset_name.." store_history=False silent=False shell_futures=True cell_id=None> result=None>

# Uploading a pre-trained Model

If you have already trained a .pt model using the airockchip/ultralytics_yolo11 repo, then on the right panel of kaggle, press upload in the input section and upload your model.

Right click on the uploaded model and copy the path, set `src_path` equal to the copied path in the code cell below before running it.

In [ ]:
import shutil

src_path = None #r"/kaggle/input/reefscape-frc-2-yolo11n-200-epochs/pytorch/default/1/Reefscape-FRC-2-640-640-yolo11n.pt"
dst_path = r"/kaggle/working/"

if src_path != None:
    shutil.copy(src_path, dst_path)

    model_name = src_path.split("/")
    model_name = model_name[len(model_name) - 1]
    model_name = model_name[:-3]

    latest = f"{root_path}/{model_name}.pt"

# Validation

Arguments for validation here: https://docs.ultralytics.com/modes/val/#arguments-for-yolo-model-validation

In [ ]:
!yolo task=detect mode=val model={latest} data={dataset.location}/data.yaml device = cuda:0,1 batch = 200

Ultralytics 8.3.9 🚀 Python-3.11.12 torch-2.6.0+cu124 
Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
             ^^^^^^^^^^^^
  File "/content/ultralytics/ultralytics/cfg/__init__.py", line 832, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/ultralytics/ultralytics/engine/model.py", line 635, in val
    validator(model=self.model)
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/content/ultralytics/ultralytics/engine/validator.py", line 127, in __call__
    device=select_device(self.args.device, self.args.batch),
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/ultralytics/ultralytics/utils/torch_utils.py", line 192, in select_device
    raise ValueError(
ValueError: Invalid CU

# RKNN Conversion

## ONNX Conversion

Intermediate step between PyTorch models and RKNN models. ONNX is another type of model. ONNX or Open Neural Network Exchange is used so that models can be used across different frameworks, operating systems, and devices.

See the Ultralytics YOLO Docs to learn the [arguments](https://docs.ultralytics.com/modes/export/#arguments) and [export formats](https://docs.ultralytics.com/modes/export/#export-formats) of the export function.

In [ ]:
#exports a differently formatted model
%cd {root_path}/ultralytics
!pip install onnx
!yolo mode=export format=rknn model={latest}


/content/ultralytics
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 112.5 MB/s eta 0:00:00
Ultralytics 8.3.9 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
custom_YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from '/content/Crime_Detection_1-448-448-yolov11n.pt' with input shape (1, 3, 448, 448) BCHW and output shape(s) ((1, 64, 56, 56), (1, 2, 56, 56), (1, 1, 56, 56), (1, 64, 28, 28), (1, 2, 28, 28), (1, 1, 28, 28), (1, 64, 14, 14), (1, 2, 14, 14), (1, 1, 14, 14)) (5.2 MB)

RKNN: starting export with torch 2.6.0+cu124...

RKNN: feed /content/Crime_Detection_1-448-448-yolov11n.onnx to RKNN-Toolkit or RKNN-Toolkit2 to generate RKNN model.
Refer https://github.com/airockchip/rknn_model_zoo/tree/main/examples/
RKNN: export success ✅ 0.8s, saved as '/content/Crime_Detection_1-448-448-yolov11n.onnx' (9.9 MB)

Export complete (2.4s)
Results saved to /content
Predict:         yolo predict task=detect model=/content

In [ ]:
#gets the path of the new onnx model, which is the same as the previous model, best.pt. The new model is called best.onnx
ex_path = '.'.join(latest.split('.')[:-1]) + '.onnx'
print(ex_path)

/content/Crime_Detection_1-448-448-yolov11n.onnx


## Installing RKNN Toolkit
This is the 3rd party toolkit that allows us to convert YOLO11 models from official ultralytics formats to the RKNN format that's used by
Rockchips.

Check the [RKNN Toolkit 2 ReadMe](https://github.com/rockchip-linux/rknn-toolkit2/blob/master/README.md) to see if it can support your NPU's platform. For example, the RKNN Toolkit 2 can support an Orange Pi 5's NPU Platform: RK3588.

In [1]:
!wget https://github.com/airockchip/rknn-toolkit2/raw/refs/heads/master/rknn-toolkit2/packages/x86_64/rknn_toolkit2-2.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install ./rknn_toolkit2-2.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

%cd {root_path}

--2025-04-17 21:36:21--  https://github.com/airockchip/rknn-toolkit2/raw/refs/heads/master/rknn-toolkit2/packages/x86_64/rknn_toolkit2-2.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/airockchip/rknn-toolkit2/refs/heads/master/rknn-toolkit2/packages/x86_64/rknn_toolkit2-2.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl [following]
--2025-04-17 21:36:22--  https://raw.githubusercontent.com/airockchip/rknn-toolkit2/refs/heads/master/rknn-toolkit2/packages/x86_64/rknn_toolkit2-2.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443.

## Setting Up RKNN Model Zoo 🦓

In [ ]:
%cd {root_path}
!git clone https://github.com/airockchip/rknn_model_zoo/
%cd rknn_model_zoo
%cd examples/yolo11/python

[Errno 2] No such file or directory: '{root_path}'
/content
Cloning into 'rknn_model_zoo'...
remote: Enumerating objects: 3202, done.
remote: Counting objects: 100% (678/678), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 3202 (delta 483), reused 488 (delta 422), pack-reused 2524 (from 2)
Receiving objects: 100% (3202/3202), 286.37 MiB | 15.56 MiB/s, done.
Resolving deltas: 100% (1094/1094), done.


In [ ]:
%%writefile imgs.txt
imgs/1.jpg
imgs/2.jpg
imgs/3.jpg
imgs/4.jpg
imgs/5.jpg
imgs/6.jpg
imgs/7.jpg
imgs/8.jpg
imgs/9.jpg
imgs/10.jpg
imgs/11.jpg
imgs/12.jpg
imgs/13.jpg
imgs/14.jpg
imgs/15.jpg
imgs/16.jpg
imgs/17.jpg
imgs/18.jpg
imgs/19.jpg
imgs/20.jpg

Writing imgs.txt


In [ ]:
import os
import shutil
import random
import glob

def copy_and_rename_images(source_folder, destination_folder, n):
    if not os.path.exists(source_folder):
        print(f"Source folder '{source_folder}' does not exist.")
        return
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    image_files = glob.glob(os.path.join(source_folder, '*.jpg'))
    selected_images = random.sample(image_files, min(n, len(image_files)))
    for i, image_path in enumerate(selected_images, start=1):
        destination_path = os.path.join(destination_folder, f'{i}.jpg')
        shutil.copy(image_path, destination_path)
    print(f"{min(n, len(image_files))} random images copied from '{source_folder}' to '{destination_folder}' and renamed.")
#putting images from dataset into imgs file
copy_and_rename_images(dataset.location+"/test/images" , "imgs", 20)

20 random images copied from '/content/ultralytics/Crime-Detection-1/test/images' to 'imgs' and renamed.


In [ ]:
#Goes to the file in line. Everything else in the cell is a list[] of 2 element tuples()
@register_line_cell_magic
def replaceAllInFile(line, cell):
    filename = line.strip()
    replacements = eval(cell)  # Assuming input is a valid Python expression
    with open(filename, 'r') as f:
        file_content = f.read()
    for replaced, with_this in replacements: #for every tuple in the list, takes the first element of the tuple in the file, and replaces it with the second element of the tuple
        file_content = re.sub(replaced, with_this, file_content)
    with open(filename, 'w') as f:
        f.write(file_content)
    print(f"Replaced successfully in {filename}")

In [ ]:
%%replaceAllInFile {root_path}/rknn_model_zoo/examples/yolo11/python/convert.py
[
    ('../../../datasets/COCO/coco_subset_20.txt', 'imgs.txt'),
]

Replaced successfully in /content/rknn_model_zoo/examples/yolo11/python/convert.py


## Quantization

Here you choose whether to perform quantization, which makes the model lighter and faster, by converting all 32/16 bit floates in the model into 8 bit ints, which costs performance.

In [ ]:
to_quantize = True

## Exporting to RKNN
### You did it!!!! 👏
Find the new model which ends in .rknn in the file directories and download it to your laptop. You now possess your own YOLO11 model that can run on an Orange Pi 5.

`!python convert.py <ONNX Model> <platform> *optional<dtype> *optional<output_model_path>`

* `<onnx_model>`: Specify ONNX model path.
* `<TARGET_PLATFORM>`: Specify NPU platform name.
  * Platforms: RK3566 | RK3568 | RK3588 | RK3562 | RK3576 | RV1103 | RV1106 | RK1808 | RK3399PRO | RV1109 | RV1126
* `<dtype>`: i8/u8 for quantization, fp for no quantization. Default is i8/u8.
* `<output_rknn_path>`: Specify save path for the RKNN model, default save in the same directory as ONNX model with name yolo11.rknn

In [ ]:
%cd {root_path}/rknn_model_zoo/examples/yolo11/python
quant_code = "i8" if to_quantize else "fp"
image_size = 640

#path of outputted rknn model as a string variable
output_model = f"{root_path}/{model_name}.rknn"
print(f"RKNN model name: {model_name}.rknn")

!python convert.py {ex_path} rk3588 {quant_code} {output_model}

/content/rknn_model_zoo/examples/yolo11/python
RKNN model name: Crime_Detection_1-448-448-yolov11n.rknn
Traceback (most recent call last):
  File "/content/rknn_model_zoo/examples/yolo11/python/convert.py", line 2, in <module>
    from rknn.api import RKNN
ModuleNotFoundError: No module named 'rknn'


# Deploying Code on PhotonVision for FRC

Only works with an Orange Pi 5. Flash a micro SD card with an PhotonVision Orange Pi image using a software like Balena Etcher. Place the micro SD card in the Pi before powering it up. Connect the Pi to your robot's radio using ethernet. Then connect to your robot's radio and type in photonvision.local:5800 in the URL of your search engine.

### Official PhotonVision release

[PhotonVision Docs](https://docs.photonvision.org/en/latest/)

Run the code cells below to output a labels.txt file which is needed when uploading your own custom model. You can use PhotonVision's UI to upload your .rknn model and labels.txt file

In [ ]:
%cd {root_path}

labels_file = f"{model_name}-labels.txt"

import yaml

with open(dataset.location + "/data.yaml", 'r') as stream:
    data = yaml.safe_load(stream)
    with open(f"{root_path}/{labels_file}", "w") as file:
        for index, label in enumerate(data['names']):
            if index == len(data['names']) - 1:
                file.write(f"{label}")
            else:
                file.write(f"{label}\n")

print(f"Successfully created {labels_file}")

/content
Successfully created Crime_Detection_1-448-448-yolov11n-labels.txt


### RKNN Fork

PhotonVision RKNN Fork Setup Instructions here: https://github.com/laviRZ/photonvision/blob/master/rknn-readme.md.

# YOLO11 Inference
You can use [this notebook](https://www.kaggle.com/code/vrishabmakam/frog-force-503-yolo11-inference) to inference a video or image with a PyTorch(.pt) YOLO11 model.